<a href="https://colab.research.google.com/github/mijanr/TimeSeries/blob/master/Ford_data_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [270]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
#1d interpolation
from scipy.interpolate import interp1d
from tslearn.datasets import UCR_UEA_datasets

#tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/ford_data_classification')

In [271]:
dataset_name = 'FordA'

In [272]:
#load dataset using tslearn
data_loader = UCR_UEA_datasets()
X_train, y_train, X_test, y_test  = data_loader.load_dataset(dataset_name)

In [273]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3601, 500, 1), (3601,), (1320, 500, 1), (1320,))

In [274]:
#convert y to 0 and 1
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [275]:
# #normalize data
# X_train = (X_train - X_train.min()) / (X_train.max() - X_train.min())
# X_test = (X_test - X_test.min()) / (X_test.max() - X_test.min())

In [276]:
#interpolate data
length = 100
def interpolate(X, length):
    X_interpolated = []
    for i in range(X.shape[0]):
        x = np.linspace(0, 1, X.shape[1])
        f = interp1d(x, X[i], axis=0)
        xnew = np.linspace(0, 1, length)
        ynew = f(xnew)
        X_interpolated.append(ynew)
    return np.array(X_interpolated)

In [277]:
X_train = interpolate(X_train, length)
X_test = interpolate(X_test, length)

In [278]:
#convert to torch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

In [279]:
#convert to torch dataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
#convert to dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [280]:
#check shapes
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 64, 1]) torch.Size([64])


In [281]:
#1D CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.sequential_1 = nn.Sequential(
            nn.Conv1d(1, 64, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(128, 128, 3),
            nn.ReLU(),
            #global average pooling
            nn.AdaptiveAvgPool1d(1)
        )
        self.sequential_2 = nn.Sequential(
            nn.LazyLinear(out_features=128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.sequential_1(x)
        x = x.view(x.size(0), -1)
        x = self.sequential_2(x)
        return x
#LSTM
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=3, batch_first=True)
        self.sequential = nn.Linear(64, 2)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.sequential(x)
        return nn.functional.log_softmax(x, dim=1)
    

In [282]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [283]:
#CNN
model = CNN().to(device)
#LSTM
#model = LSTM().to(device)

In [284]:
#feed a sample to the model
for x, y in train_loader:
    x = x.to(device)
    print(model(x).shape)
    break

torch.Size([64, 2])


In [285]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [286]:
#training
n_epochs = 100
for epoch in range(n_epochs):
    for i, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    #print loss
    if epoch % 10 == 0:
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, loss.item()))
        #writer.add_scalar('training loss', loss.item(), epoch)
 

Epoch: 0 	Training Loss: 0.543900
Epoch: 10 	Training Loss: 0.350338
Epoch: 20 	Training Loss: 0.206480
Epoch: 30 	Training Loss: 0.016042
Epoch: 40 	Training Loss: 0.041306
Epoch: 50 	Training Loss: 0.000032
Epoch: 60 	Training Loss: 0.000020
Epoch: 70 	Training Loss: 0.000011
Epoch: 80 	Training Loss: 0.000009
Epoch: 90 	Training Loss: 0.000000


In [287]:
#check the mdodel on test data
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
    print(f'Accuracy: {100 * correct / total} %')

Accuracy: 90.68181818181819 %
